### Importación de los datos

In [1]:
## Se importa las librerías necesarias
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import pandas as pd

## Se importa el dataset con latencia de 7 días
Data = read_csv('data_7dias_latencia.csv', delimiter=',', encoding='cp1252')

## Se elimina las columnas que no son utilizadas en el análisis
del Data['state']
del Data['station']
del Data['date']

## Se almacena en una lista el nombre de las columnas del dataset
col_names=list(Data.columns)

## Se normaliza los datos
mm_scaler=preprocessing.MinMaxScaler()
df_1=mm_scaler.fit_transform(Data)
df=pd.DataFrame(df_1,columns=col_names)

df.head()

,ws,air_temp,visibility,RH,new_cases
0,0.562026,0.807566,0.361702,0.729768,0.094703
1,0.442043,0.191159,0.531915,0.681431,0.096308
2,0.437254,0.106840,0.210402,0.797240,0.891921
3,0.390664,0.814246,0.316306,0.685663,0.121990
4,0.390664,0.814246,0.316306,0.685663,0.020867


### Red neuronal (perceptrón multicapa) con el algoritmo PSO

In [2]:
## Se importa las librerías necesarias
import functools
import numpy as np
import sklearn.metrics
import sklearn.model_selection

from pso import pso, ann


def dim_weights(shape):
    dim = 0
    for i in range(len(shape)-1):
        dim = dim + (shape[i] + 1) * shape[i+1]
    return dim

def weights_to_vector(weights):
    w = np.asarray([])
    for i in range(len(weights)):
        v = weights[i].flatten()
        w = np.append(w, v)
    return w

def vector_to_weights(vector, shape):
    weights = []
    idx = 0
    for i in range(len(shape)-1):
        r = shape[i+1]
        c = shape[i] + 1
        idx_min = idx
        idx_max = idx + r*c
        W = vector[idx_min:idx_max].reshape(r,c)
        weights.append(W)
    return weights

def eval_neural_network(weights, shape, X, y):
    mse = np.asarray([])
    for w in weights:
        weights = vector_to_weights(w, shape)
        nn = ann.MultiLayerPerceptron(shape, weights=weights)
        y_pred = nn.run(X)
        mse = np.append(mse, sklearn.metrics.mean_squared_error(np.atleast_2d(y), y_pred))
    return mse

def print_best_particle(best_particle):
    print("Nueva mejor partícula encontrada en la iteración #{i} con mean squared error: {score}"\
          .format(i=best_particle[0], score=best_particle[1]))


## Número de clases
num_classes = 2

## Se divide el dataset normalizado en: variables independientes y variable dependiente (clase)
data_array = df.values  

data_x = data_array[:,0:4]     # Variables independientes
data_y = data_array[:,4]       # Variable dependiente (clase)

## Se crea la clase binaria
label_y = np.where(data_y <= 0.0695, 
                   0,     # 0: Número bajo de casos covid-19
                   1)     # 1: Número alto de casos covid-19
    
## Se define los conjuntos de datos para entrenamiento y prueba   
X, X_test, y, y_test = sklearn.model_selection.train_test_split(data_x, label_y
                                                               ,test_size=0.34   # 66% datos para entrenar; 34% datos para validación
                                                                ,random_state=0)

num_inputs = X.shape[1]

# Se construye una extructura matricial para el conjunto de datos de entrenamiento
y_true = np.zeros((len(y), num_classes))
for i in range(len(y)):
    y_true[i, y[i]] = 1
    
# Se construye una estructura matricial para el conjunto de datos de prueba
y_test_true = np.zeros((len(y_test), num_classes))
for i in range(len(y_test)):
    y_test_true[i, y_test[i]] = 1

## Construcción del modelo.........................
shape = (num_inputs,   # capa de entrada con 4 neuronas
         64,           # capa intermedia 1 con 64 neuronas
         32,           # capa intermedia 2 con 32 neuronas
         num_classes)  # capa de salida con 2 neuronas


cost_func = functools.partial(eval_neural_network, shape=shape, X=X, y=y_true.T)

swarm = pso.ParticleSwarm(cost_func,  # función de costo de la red neuronal 
                          num_dimensions=dim_weights(shape), # número de dimensiones 
                          num_particles=10,    # número de particulas 32
                          chi=0.72984,         # constante inercial del peso (cuanto es el peso de la velocidad previa)
                          phi_p=2.05,          # constante cognitiva 1.49
                          phi_g=2.05)          # constante global

# Entrenamieno del modelo..........................
i = 0
best_scores = [(i, swarm.best_score)]
print_best_particle(best_scores[-1])
while swarm.best_score>1e-6 and i<350:  
    swarm._update()
    i = i+1
    if swarm.best_score < best_scores[-1][1]:
        best_scores.append((i, swarm.best_score))
        print_best_particle(best_scores[-1])

best_weights = vector_to_weights(swarm.g, shape)
best_nn = ann.MultiLayerPerceptron(shape, weights=best_weights)

# Predicción del modelo...........................
y_test_pred = np.round(best_nn.run(X_test))


Nueva mejor partícula encontrada en la iteración #0 con mean squared error: 0.4999993498795848
Nueva mejor partícula encontrada en la iteración #3 con mean squared error: 0.49675298525284833
Nueva mejor partícula encontrada en la iteración #4 con mean squared error: 0.242730794093153
Nueva mejor partícula encontrada en la iteración #5 con mean squared error: 0.20624848793179268
Nueva mejor partícula encontrada en la iteración #6 con mean squared error: 0.20580769113285413
Nueva mejor partícula encontrada en la iteración #7 con mean squared error: 0.20506966319492195
Nueva mejor partícula encontrada en la iteración #10 con mean squared error: 0.20351971562510818
Nueva mejor partícula encontrada en la iteración #11 con mean squared error: 0.2028682146374755
Nueva mejor partícula encontrada en la iteración #14 con mean squared error: 0.20177535915066092
Nueva mejor partícula encontrada en la iteración #29 con mean squared error: 0.20154296436026486
Nueva mejor partícula encontrada en la i

Nueva mejor partícula encontrada en la iteración #307 con mean squared error: 0.18100388153508676
Nueva mejor partícula encontrada en la iteración #308 con mean squared error: 0.1809764626292477
Nueva mejor partícula encontrada en la iteración #309 con mean squared error: 0.18094475718589167
Nueva mejor partícula encontrada en la iteración #310 con mean squared error: 0.1809229107444428
Nueva mejor partícula encontrada en la iteración #311 con mean squared error: 0.1808994172864838
Nueva mejor partícula encontrada en la iteración #312 con mean squared error: 0.18087580052911456
Nueva mejor partícula encontrada en la iteración #313 con mean squared error: 0.1808530635145313
Nueva mejor partícula encontrada en la iteración #314 con mean squared error: 0.18082975208937646
Nueva mejor partícula encontrada en la iteración #315 con mean squared error: 0.18081017179741857
Nueva mejor partícula encontrada en la iteración #316 con mean squared error: 0.18079169584717572
Nueva mejor partícula en

### Métricas de evaluación del modelo 

In [3]:
print(sklearn.metrics.classification_report(y_test_true, y_test_pred.T))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83       119
           1       0.33      0.02      0.04        49

   micro avg       0.71      0.71      0.71       168
   macro avg       0.52      0.51      0.44       168
weighted avg       0.60      0.71      0.60       168
 samples avg       0.71      0.71      0.71       168



#### Accuracy

In [4]:
print(sklearn.metrics.accuracy_score(y_test_true, y_test_pred.T))

0.7023809523809523


#### ROC AUC

In [5]:
print(sklearn.metrics.roc_auc_score(y_test_true, y_test_pred.T))

0.5060024009603842


#### Matriz de clasificación

In [7]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test_true, y_test_pred.T, labels=[0, 1])

array([[[  1,  48],
        [  5, 114]],

       [[115,   4],
        [ 47,   2]]], dtype=int64)